This is just a simple translation using tensorflow

In [29]:
!pip install tensorflow
!pip install tensorflow_hub

In [30]:
import tensorflow as tf
import tensorflow_hub as hub

In [31]:
# Load the pre-trained model
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [32]:
# Prepare the input text
input_text = tf.constant(["This is a test sentence."])

In [33]:
# Translate the input text
translated_text = model(input_text)

In [34]:
# Print the translated text
print(translated_text)

tf.Tensor(
[[ 4.74886075e-02 -4.98664305e-02 -1.71142798e-02  2.99241040e-02
  -5.26666492e-02 -5.95821179e-02  6.11432735e-03  3.35058942e-02
   4.44955453e-02  3.38426754e-02  1.85408853e-02 -4.16645296e-02
   7.37075880e-02  5.72554395e-02  1.80599913e-02 -7.43023753e-02
   2.06891284e-03  4.64502871e-02  4.52925861e-02 -8.28705132e-02
  -1.68106088e-03  5.97309731e-02 -6.28478751e-02  1.11337891e-02
  -6.62655532e-02  3.79015058e-02 -9.23611596e-03 -5.67900389e-02
   1.12583283e-02 -5.01601472e-02  5.57581894e-02 -4.20861840e-02
   1.50783211e-02 -1.50357671e-02 -6.93174526e-02 -1.72347855e-03
   7.31806410e-03 -2.83883624e-02  3.56385820e-02  6.78013265e-02
  -2.56217755e-02  1.15059121e-02  1.65836699e-02  5.62694445e-02
  -3.36205587e-02 -9.90155619e-03 -1.87957827e-02 -1.58994086e-02
  -6.90774701e-04  1.12931561e-02  4.27371860e-02 -6.07277751e-02
  -9.51165557e-02 -2.31154151e-02  1.43807447e-02 -4.02030163e-02
   6.97867423e-02 -3.61778662e-02 -3.99746299e-02  4.51778844e-02

To implement a simple neural machine translation (NMT) model in Python, we can use the TensorFlow and Keras libraries. TensorFlow is a popular deep learning framework that provides tools for building and training neural networks, while Keras is a high-level neural networks API that runs on top of TensorFlow.

Here's a step-by-step explanation and implementation of a simple NMT model using TensorFlow and Keras:

In [69]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

Prepare Data: We need a dataset containing pairs of sentences in two languages (source and target). For simplicity, we'll use dummy data.

In [39]:
# Dummy data
input_texts = ['hello', 'good morning', 'how are you']
target_texts = ['bonjour', 'bonjour', 'comment ça va']

# Generate vocabulary
input_vocab = set()
target_vocab = set()
for input_text, target_text in zip(input_texts, target_texts):
    input_vocab.update(input_text)
    target_vocab.update(target_text)

input_vocab = sorted(input_vocab)
target_vocab = sorted(target_vocab)

# Add special tokens for padding and start/end of sequence
input_vocab_size = len(input_vocab) + 2
target_vocab_size = len(target_vocab) + 2

# Create dictionaries to map characters to indices and vice versa
input_token_index = dict([(char, i+1) for i, char in enumerate(input_vocab)])
target_token_index = dict([(char, i+1) for i, char in enumerate(target_vocab)])

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

# Define maximum sequence lengths
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# Initialize arrays for encoder and decoder inputs
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, input_vocab_size), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, target_vocab_size), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, target_vocab_size), dtype='float32')

# Fill the arrays with one-hot encoding
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0


Define the Model: We'll create a simple sequence-to-sequence model with an LSTM encoder and decoder.

In [40]:
# Define input sequence
encoder_inputs = Input(shape=(None, input_vocab_size))
# LSTM encoding
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Discard encoder outputs, we only need the states
encoder_states = [state_h, state_c]

# Set up the decoder, using encoder_states as initial state
decoder_inputs = Input(shape=(None, target_vocab_size))
# We set up our decoder to return full output sequences and to return internal states as well
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


Compile and Train the Model: Compile the model with appropriate loss function and optimizer, then fit the model to the data.

In [41]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=1, epochs=50, validation_split=0.2)


Epoch 1/50
2/2 [==============================] - 4s 1s/step - loss: 1.2676 - val_loss: 2.5636
Epoch 2/50
2/2 [==============================] - 0s 91ms/step - loss: 1.2222 - val_loss: 2.5753
Epoch 3/50
2/2 [==============================] - 0s 97ms/step - loss: 1.1608 - val_loss: 2.6765
Epoch 4/50
2/2 [==============================] - 0s 105ms/step - loss: 0.9860 - val_loss: 2.9888
Epoch 5/50
2/2 [==============================] - 0s 88ms/step - loss: 0.7581 - val_loss: 2.9783
Epoch 6/50
2/2 [==============================] - 0s 98ms/step - loss: 0.6612 - val_loss: 3.0365
Epoch 7/50
2/2 [==============================] - 0s 110ms/step - loss: 0.5937 - val_loss: 3.2744
Epoch 8/50
2/2 [==============================] - 0s 120ms/step - loss: 0.5306 - val_loss: 3.1573
Epoch 9/50
2/2 [==============================] - 0s 145ms/step - loss: 0.5890 - val_loss: 3.5419
Epoch 10/50
2/2 [==============================] - 0s 60ms/step - loss: 0.4633 - val_loss: 3.5553
Epoch 11/50
2/2 [==========

Inference: To test the model, we need to set up the inference mode separately.

In [42]:
# Define encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Define decoder model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [47]:
print(target_token_index)

{' ': 1, 'a': 2, 'b': 3, 'c': 4, 'e': 5, 'j': 6, 'm': 7, 'n': 8, 'o': 9, 'r': 10, 't': 11, 'u': 12, 'v': 13, 'ç': 14}


In [48]:
if '\t' not in target_token_index:
    print('The character "\t" is not present in the target_token_index dictionary.')

The character "	" is not present in the target_token_index dictionary.


In [49]:
target_token_index['\t'] = len(target_token_index) + 1

To evaluate the performance and check the accuracy of the neural machine translation (NMT) model, we can perform inference on the test data and calculate the accuracy based on the model's predictions compared to the ground truth translations.

In [51]:
# Define a function to decode sequences
def decode_sequence(input_seq):
    # Encode the input sequence to get the internal state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, target_vocab_size))
    # Populate the first character of target sequence with the start character
    target_seq[0, 0, target_token_index['\t']] = 1.0

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (length 1).
        target_seq = np.zeros((1, 1, target_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Define test data
test_input_texts = ['hello', 'good morning', 'how are you']
test_target_texts = ['bonjour', 'bonjour', 'comment ça va']

# Initialize variables for accuracy calculation
total_samples = len(test_input_texts)
correct_predictions = 0

# Iterate through test data
for input_text, target_text in zip(test_input_texts, test_target_texts):
    # Convert input sequence to one-hot encoding
    encoder_input_data = np.zeros((1, max_encoder_seq_length, input_vocab_size), dtype='float32')
    for t, char in enumerate(input_text):
        encoder_input_data[0, t, input_token_index[char]] = 1.0

    # Decode the input sequence
    decoded_sentence = decode_sequence(encoder_input_data)

    # Print the input, target, and predicted translations
    print('Input:', input_text)
    print('Target:', target_text)
    print('Predicted:', decoded_sentence)

    # Update accuracy count
    if decoded_sentence.strip() == target_text.strip():
        correct_predictions += 1

# Calculate accuracy
accuracy = (correct_predictions / total_samples) * 100
print('Accuracy:', accuracy, '%')

1/1 [==============================] - 0s 22ms/step
Input: hello
Target: bonjour
Predicted: onjouroooooooo
1/1 [==============================] - 0s 25ms/step
Input: good morning
Target: bonjour
Predicted: onjouroooooooo
1/1 [==============================] - 0s 24ms/step
Input: how are you
Target: comment ça va
Predicted: onjouroooooooo
Accuracy: 0.0 %


This code will perform inference on the test data, printing the input, target, and predicted translations for each example. Finally, it will calculate and print the accuracy of the model's translations compared to the ground truth translations.

Ensure that the model has been trained properly on suitable data and adjust the code accordingly if your dataset or model architecture differs.

2nd Part :-
To implement another simple neural machine translation (NMT) model in Python, we can use the PyTorch library. PyTorch is a popular deep learning framework known for its flexibility and ease of use. Create a basic sequence-to-sequence model with an encoder and decoder architecture.

Here's the step-by-step explanation and implementation:

Import Libraries: We start by importing the necessary libraries.

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

Prepare Data: We need a dataset containing pairs of sentences in two languages (source and target). For simplicity, use dummy data similar to the previous example.

In [65]:
# Dummy data
input_texts = ['hello', 'good morning', 'how are you']
target_texts = ['bonjour', 'bonjour', 'comment ça va']

# Generate vocabulary
input_vocab = set()
target_vocab = set()
for input_text, target_text in zip(input_texts, target_texts):
    input_vocab.update(input_text)
    target_vocab.update(target_text)

input_vocab = sorted(input_vocab)
target_vocab = sorted(target_vocab)

# Add special tokens for padding and start/end of sequence
input_vocab_size = len(input_vocab) + 2
target_vocab_size = len(target_vocab) + 2

# Create dictionaries to map characters to indices and vice versa
input_token_index = dict([(char, i+1) for i, char in enumerate(input_vocab)])
target_token_index = dict([(char, i+1) for i, char in enumerate(target_vocab)])

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

# Define maximum sequence lengths
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# Initialize arrays for encoder and decoder inputs
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, input_vocab_size), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, target_vocab_size), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, target_vocab_size), dtype='float32')

# Fill the arrays with one-hot encoding
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

Define the Model: Create a simple sequence-to-sequence model with an encoder and decoder architecture using PyTorch.

In [66]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded)
        return output, hidden

class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = nn.functional.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

Training the Model: Need to define functions for training the model.

In [67]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=max_decoder_seq_length):
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]])

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = tensorsFromPair(random.choice(pairs))
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('(%d %d%%) %.4f' % (iter, iter / n_iters * 100, print_loss_avg))

Inference: Need to define functions for inference and evaluation.

In [68]:
def evaluate(encoder, decoder, sentence, max_length=max_decoder_seq_length):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]])
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

Conclusion: This implementation provides a basic understanding of how an NMT model works using a sequence-to-sequence architecture with an encoder and decoder. To evaluate the model's performance, we can perform inference on the test data, compare the predicted translations with the ground truth translations, and compute the error rate or accuracy.

Now, to perform inference and check the accuracy by computing the error rate, you need to train the model using the provided training data, then evaluate it on the test data. You'll compare the model's predictions with the actual translations and calculate the accuracy accordingly.

Ensure that the model is trained properly and adjust the code accordingly if your dataset or model architecture differs.

In [74]:
!ls

sample_data


In [76]:
import os
print(os.getcwd())

/content


In [77]:
!pip show torch

Name: torch
Version: 2.1.0+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision
